In [2]:
%load_ext autoreload
%autoreload 2

import os
import glob
import json
import numpy as np
import pandas as pd
import sqlite3 as sql
from collections import defaultdict

import matplotlib.pyplot as plt
import seaborn as sns

from grid_search_graph import generate_plot

In [11]:
base_dir = "/tmp/grid_search"

results = glob.glob(os.path.join(base_dir, "*", "results.json"))

data = []
for i, r in enumerate(sorted(results)):
    with open(r, 'r') as f:
        j = json.load(f)
    j["nn_fps"] = int(np.mean(j["nn_fps"]))
    for k, v in j["config"].items():
        j[k] = v
    del j["config"]

    maps = defaultdict(int)
    for e, m in j['mAPs']:
        for th, v in m.items():
            maps[th] = max(maps[th], v)
    del j['mAPs']
    for th, v in maps.items():
        j["mAP@{}".format(th)] = v

    stats = {"TP": {}, "FP": {}, "FN": {}}
    e, (tp, fp, fn) = j["stats"][-1]
    for (th, tpv), fpv, fnv in zip(tp.items(), fp.values(), fn.values()):
        stats["TP"][th] = tpv
        stats["FP"][th] = fpv
        stats["FN"][th] = fnv
    del j['stats']
    for name, values in stats.items():
        for th, v in values.items():
            j["{}@{}".format(name, th)] = v
    j["last_epoch"] = e
    j["test"] = i
    data.append(j.values())

print([type(v) for v in j.values()])
df = pd.DataFrame(data, columns=j.keys())

con = sql.Connection(":memory:")

df.to_sql("glob_stats", con)

print(len(df))

[<class 'int'>, <class 'int'>, <class 'int'>, <class 'dict'>, <class 'float'>, <class 'str'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>]


InterfaceError: Error binding parameter 4 - probably unsupported type.